# OverRepresentationAnalysis
## Formalisme statitisque et mise en forme de résultats

##### IFB Virtual Machine Configuration

* `rootDir` points to Jupyter Home Directory
* `projectDir` points to the Git project, holds the *tsv* file
* `libDir` points to Git project python library with files names `go.py  stat_utils.py  uniprot.py`
* `dataDir` points to shared data folder with uniprot and GO files

In [3]:
import sys

rootDir = "//ifb/data/mydatalocal"
projectDir = rootDir + "/VDBscratch"
libDir = projectDir + "/lib"
dataDir= "/ifb/data/public/teachdata/m2binf_ucbl/data" 

sys.path.append(libDir)

In [4]:
import pandas, uniprot, go
import numpy as np
from stat_utils import computeORA_BKG

%load_ext autoreload
%autoreload 2

In [5]:
%matplotlib nbagg
import matplotlib.pyplot as plt

## Jeux de données
* Experimental dataset as TSV
* Go ontology as [owl](http://www.obofoundry.org/ontology/go.html)
* Uniprot entries as XML files

## Objectifs
1. Lire et traiter des données d'entrée avec [PANDAS](https://pandas.pydata.org/)

2. Représenter la distribution empirique des "fold-change" (**FC**)

3. Evaluer un seuil de **FDR** 5%, with normal approximation
    1. Proposer une function qui à un seuil de faux-positif retourne une valeur de **FDC**
    2. Proposer une fonction qui appelle la précedente et retourne, en plus, une représentation graphique

4. Analyse de l'enrichissement en pathway Gene Ontology
    1. Arbre des termes d'ontologie.
        * Visualisation de l'arbre complet
    2. Modélisation hypergéometrique des fréquence attendues des termes GO
        * Les protéines très abondantes présentent-elles des termes GO en quantité remarquable ?
        * Les protéines peu abondantes présentent-elles des termes GO en quantité remarquable ?
    3. Visualisation de l'arbre avec style des noeuds rendant compte des protéines qu'ils annotent
 
5. Volcano plot
    1. Scatter-Plot **FC** vs **LOG10 Adj.P-val**
    2. Colorer les protéines très/peu abondantes
    3. ipyWidget slider FDR permettant de changer les seuils "d'abondance" 
    
6. Volcano Plot présentant les pathways
    1. ipyWidget Slider
    2. Coloration des points pour les protéines annotées par ce pathway 

## Conclusion
* Compare wt/mutant conditions


## Ressources
https://www.bioconductor.org/help/course-materials/2015/SeattleApr2015/E_GeneSetEnrichment.html

https://matplotlib.org/gallery.html#




## Format des données expérimentales

Le terme *SA*, désigne le ratio de (**S**ur) **A**bondance pour une protéines entre deux experiences.
Les valeurs de **SA** sont stockées dans la colonne ```Log2 Corrected Abundance Ratio```. Un "ratio" négatif indique qu'une protéine est moins présente dans l'experience considéré par rapport à une experience de référence.

* La condition experimentale de référence est un milieu de croissance bactérienne
* La condition experimentale testée est un milieu riche en Tétracycline 



# Première exploration des données

### Lecture des valeurs experimentales
Charger ```TCL_wt1.tsv``` dans une [dataframe pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
* Retirer les lignes présentant des aberrations numériques
* Créer une DataFrame ne contenant que les lignes où **Log2 Corrected Abundance Ratio** > 0

##### [Comment filtrer des lignes Pandas](https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/)

In [6]:
import pandas
df=pandas.read_csv(projectDir + "/TCL_wt1.tsv", delimiter ="\t")
df.head()

,Accession,Description,Gene Symbol,Corrected Abundance ratio (1.53),Log2 Corrected Abundance Ratio,Abundance Ratio Adj. P-Value: (127. T3 Tc WT) / (126. T0 WT),LOG10 Adj.P-val
0,P75936,Basal-body rod modification protein FlgD OS=Es...,flgD,0.075816993,-3.721334942,0.000055,4.260067469
1,P76231,Uncharacterized protein YeaC OS=Escherichia co...,yeaC,0.092810458,-3.429568818,0.000351,3.45462743
2,P0A8S9,Flagellar transcriptional regulator FlhD OS=Es...,flhD,0.102614379,-3.284695189,0.000027,4.571899347
3,P0CE48,Elongation factor Tu 2 OS=Escherichia coli (st...,tufB,#VALEUR!,#VALEUR!,NaN,#VALEUR!
4,P05706,PTS system glucitol/sorbitol-specific EIIA com...,srlB,0.108496732,-3.204276506,0.019963,1.699767669


In [10]:
dfClean = df[ (df['Log2 Corrected Abundance Ratio'] != '#VALEUR!') & (df['LOG10 Adj.P-val'] != '#VALEUR!')]
dfClean = dfClean.copy()
dfClean.loc[:,'Log2 Corrected Abundance Ratio'] = dfClean.loc[:,'Log2 Corrected Abundance Ratio'].astype(float)

saList = dfClean[ dfClean['Log2 Corrected Abundance Ratio'] > 0.0 ]['Accession'].tolist()

dfClean.head()

,Accession,Description,Gene Symbol,Corrected Abundance ratio (1.53),Log2 Corrected Abundance Ratio,Abundance Ratio Adj. P-Value: (127. T3 Tc WT) / (126. T0 WT),LOG10 Adj.P-val
0,P75936,Basal-body rod modification protein FlgD OS=Es...,flgD,0.075816993,-3.721335,0.000055,4.260067469
1,P76231,Uncharacterized protein YeaC OS=Escherichia co...,yeaC,0.092810458,-3.429569,0.000351,3.45462743
2,P0A8S9,Flagellar transcriptional regulator FlhD OS=Es...,flhD,0.102614379,-3.284695,0.000027,4.571899347
4,P05706,PTS system glucitol/sorbitol-specific EIIA com...,srlB,0.108496732,-3.204277,0.019963,1.699767669
5,P29744,Flagellar hook-associated protein 3 OS=Escheri...,flgL,0.124183007,-3.009460,0.036746,1.434786589


### Rappel
Pour faciliter le développement nous considerereons comme **S**ur**A**bondante toute protéine à **Log2 Corrected Abundance Ratio** supérieur à 0.

# Definitions de pathway fonctionels: modèle statistique

Nous disposons d'un jeu de protéines caractérisées par

* Un ratio de quantification entre deux experiences
* Une fiche uniprot 

Par ailleurs, l'ontologie GO est disponible via
* un fichier plat de définition
* une librarie pour la manipuler

Il nous est demandé de detecter si certains termes GO sont fortement représentés parmi les protéines SurAbondantes.
Par la suite, on utilisera indifférement les termes **Pathway** et **annotation GO**.

### Evaluation théorique de l'abondance

* $Q+$ : Proteines Sur-représentées
* $\overline{Q+}$ : Proteines non Sur-représentées

* $PW^{a}$ : Nombre de protéines membres du Pathway$a$
* $\overline{PW^{a}}$ : Nombre de protéines non-membres du Pathway$a$



 Proteines | $Q+$  | $\overline{Q+}$ | Total lignes
 :--- | --- |--- |---|
membres de $PW^{a}$  | $PW^{a}_{Q+}$ | $PW^{a}_{\overline{Q+}}$ | $nPW^{a}$
non-membres de $PW_{a}$ | $\overline{PW^{a}}_{Q+}$ | $\overline{PW^{a}}_{\overline{Q+}}$ | $n\overline{PW^{a}}$
  **Total colonnes** | n$Q+$ | n$\overline{Q+}$ | **nTOT**
 
 
**Question**:"Le nombre de protéines sur-représentées dans le Pathway$a$ est-il grand comparé à ce qui serait attendu au hasard?"


<!--## Deprecated 
 **Question**:"Les protéines sur-représentées sont-elles plus fréquentes dans le Pathway$a$ que dans les protéines non-membre du Pathway$X$?"
 
L'hypothèse nulle est "Les protéines membres du pathway et non-membres du pathway ont des quantifications similaires".
 -->
        

### Modélisation du hasard par une loi hypergéométrique

Pour un pathway donné ($PWa$), chaque protéine sur-abondante peut être vue comme un tirage dans le pool de protéines de l'expérience. La population de ce pool est consituée de deux groupes:
 * Le groupe des protéines appartenant à $PWa$
 * Les autres ($\overline{PW^{a}}$)

La probabilité d'observer $k$ protéines de $PWa$ parmi les $n$ sur-abondantes se modélise alors naturellement par la loi de probabilité hypergéométrique.

Rappellez-vous,

Si X est la variable aléatoire du nombre de succès parmi n observations, sans remise.
Alors la probabilité d'oberver exactement $k$ succès, suit une loi hypergéométrique de paramètres $N, n$ et $k$

$P(X=k) = \frac{ {K\choose k} {N-K\choose n-k} } { {N\choose K} }$

où


* $K$ est le nombre de succès dans la population totale
* $n$ est le nombre d'observations
* $N$ est la taille de la population totale
* $k$ est le nombre de succès parmi les observations

Si **pour chaque $Pathway^{a}$**, $X$ est la variable aléatoire du **nombre de protéines sur-représentée** dans l'experience.


Cocher les relations correspondantes


  x  | $nTOT$ | $nPW^{a}_{Q+}$ | $nPW^{a}$ | $nQ+$
 --- | ---   | ---     | ---  | --- |
 $K$ |       |         |      | 
 $n$ |       |         |      | 
 $N$ |       |         |      | 
 $k$ |       |         |      | 
 
 
 
 
Rappel: $nPW^{a}_{Q+}$ est le nombre de protéines appartenant au $Pathway^{a}$ parmi les protéines surexprimées

Que représentent alors les deux expressions suivantes:

$ P(X=nPW^{a}_{Q+})= \frac{ {nPW^{a}\choose X} {n\overline{PW^{a}}\choose nQ^{+} - X} } { {nTOT\choose nPW^{a}} }$


$P(X >= PW^{a}_{Q+})  = \sum_{x_{i}=PW^{a}_{Q+}}^{x_{i} \le nQ+}\frac{ {nPW^{a}\choose x_{i}} {n\overline{PW^{a}}\choose nQ^{+} - x_{i}} } { {nTOT\choose nPW^{a}} }$


###### Créer la fonction qui retournera la probabilité, sous un [modèle hypergeométrique](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.hypergeom.html), d'obtenir $k$ succès ou plus,

```python
def righEnd_pValue(N, n, K, k) => p
```
Aux arguments:
* $N$, le nombre total d'objet
* $n$, le nombre de tirages
* $K$, le nombre total d'objet de *typeI*
* $k$, le nombre de tirage d'objet de *typeI* (ie: nombre de succès parmi les tirages)

##### Dessiner les lollipop charts de la densité de probabilité et de la densité cumulée de la loi

L'enrichissement en terme GO parmi les protéines surabondantes sera estimé par cette fonction, avec
* $N$, le nombre total de protéines annotées dans le protéome
* $n$, le nombre de protéines surabondantes
* $K$, le nombre de protéines annotées par ce terme GO dans le protéome
* $k$, le nombre de protéines annotées par ce terme GO dans les protéines surabondantes


*Exemple de figures à obtenir*
![caption](../figs/lollipop.png)


In [ ]:
### Cellule Exercice

# Definitions de pathway fonctionels: manipulation de l'ontologie


#### Chargements des données uniprot et GO

##### Listes de protéines
Les analyses des termes GO s'opèrent sur deux ensembles (listes) de protéines différents:
1. Les protéines quantifiées dans l'experience: `xpProtList = dfClean['Accession'].tolist()`
2. [Le protéome](https://www.uniprot.org/proteomes/UP000000625) de la bactérie étudiée: `K12 = uniprot.UniprotCollection(dataDir + "/K12_proteome/")`


Le protéome est utilisé pour définir les paramètres $K$ et $N$ du modèle hypergéométrique. Que pensez-vous de ce choix ?

In [ ]:
uniprotCollection = uniprot.UniprotCollection(dataDir +"/dataset")
K12 = uniprot.UniprotCollection(dataDir + "/K12_proteome/")
xpProtList = dfClean['Accession'].tolist()

###### Ontologie
Les termes GO sont organisés en une structure hiérarchique ([DAG](https://en.wikipedia.org/wiki/Directed_acyclic_graph)). Dans ce type de graph les liens sont orientés, mais les noeuds fils peuvent avoir plusieurs parents.
Les termes de cette ontologie sont organisés en trois arbres indépendants:
* **biological process**
* **molecular function**
* **cellular component**

Prenez un moment pour vous familliariser avec [sa structure](https://www.ebi.ac.uk/QuickGO/).

* Après le chargement initial de l'ontologie complète `go.setOntology(dataDir + "/go.owl")`
* Nous travaillerons avec deux copies d'un seul arbre de terme à la fois 
    1. `xpGoTree_BP = go.AnnotationTree("biological process", collapse=True)`
    2. `fullEcoliGoTree_BP = go.AnnotationTree("biological process", collapse=True)`
* Dont nous extrairons uniquement le sous-arbre correspondant à deux ensembles de protéines 
    3. Les protéines quantifiées: `xpGoTree_BP.extract(xpProtList, uniprotCollection)`
    4. Le protéome total: `fullEcoliGoTree_BP.extract(K12.list, K12)`

In [ ]:
print("Loading ontology")
go.setOntology(dataDir + "/go.owl")

##### Décommentez les quatre commandes du namespace de votre choix

In [1]:
print("Extracting biological process ontology")
#xpGoTree_BP = go.AnnotationTree("biological process", collapse=True)
#xpGoTree_BP.extract(xpProtList, uniprotCollection)

#fullEcoliGoTree_BP = go.AnnotationTree("biological process", collapse=True)
#fullEcoliGoTree_BP.extract(K12.list, K12)

print("Extracting molecular function ontology")
#xpGoTree_MF = go.AnnotationTree("molecular function", collapse=True)
#xpGoTree_MF.extract(xpProtList, uniprotCollection)

#fullEcoliGoTree_MF = go.AnnotationTree("molecular function", collapse=True)
#fullEcoliGoTree_MF.extract(K12.list, K12)

print("Extracting cellular component ontology")
#xpGoTree_CC = go.AnnotationTree("cellular component", collapse=True)
#xpGoTree_CC.extract(xpProtList, uniprotCollection)

#fullEcoliGoTree_CC = go.AnnotationTree("cellular component", collapse=True)
#fullEcoliGoTree_CC.extract(K12.list, K12)

Extracting biological process ontology
Extracting molecular function ontology
Extracting cellular component ontology


## Manipulation de termes GO à l'aide de *go.py*

Règle du vrai chemin: *Si une protéine est porteuse d'un terme GO, alors est elle aussi porteuse de tous les parents de ce terme.*

**AnnotationTree est l'arbre des annotations GO portées par une collection de proteines**


### Un arbre ne rend compte que d'un seul namespace GO
Pour évaluer la représentation statisitique des différents termes GO, nous devons calculer les fréquences des différents termes dans deux groupes de protéines:

* Le groupe expérimental : Cette version de l'arbre recensera les protéines de l'expérience. Préfixe:  **xpGoTree_**
* Le groupe "background" : Cette version de l'arbre recensera les protéines du protéomes de référence. Préfixe: **fullEcoliGoTree_**

On définit le namespace des termes GO à extraire par un de ces trois termes : 
* "biological process", "molecular function", "cellular component"
 
```python
aTree = go.AnnotationTree("biological process", collapse=True)
```
On ne garde dans cet arbre que les termes portés par les protéines données en dernier argument
```python
aTree.extract(['P75936', 'P76231', 'P0A8S9'], uniprotCollection)
```

Dans cet arbre:
* **Les noeuds sont des termes GO**
* **Les feuilles sont des protéines portant le terme GO**


#### Exemple, avec les données de notre étude:
* Données experimentales
    - liste de protéines : *xpProtList* 
    - collection d'objet uniprot : *uniprotCollection*
* Données "background"
    - liste de protéines : *K12.list* 
    - collection d'objet uniprot : *K12*
    

```python
print("Extracting biological process ontology")
xpGoTree_BP = go.AnnotationTree("biological process", collapse=True)
xpGoTree_BP.extract(xpProtList, uniprotCollection)

fullEcoliGoTree_BP = go.AnnotationTree("biological process", collapse=True)
fullEcoliGoTree_BP.extract(K12.list, K12)
```
*fullEcoliGoTree_BP* stockera l'information d'ontologie pour tout le protéome d' E.Coli. Tandis que *xpGoTree_BP* stockera l'information d'ontologie pour les protéines de l'experience.

### API de l'arbre d'annotations


#### Taille de l'abre
Affiche un tuple (feuilles, noeuds, protéines)
```python
aTree.dimensions
```
example:
```python
xpGoTree_BP.dimensions
(2194, 3482, 1474)
```

#### Extraire un sous arbre
```python
bTree = aTree.newRoot("transmembrane transport")
```

#### Enumerer les protéines porteuses de termes GO

##### Dans tout l'arbre
```python
proteinList = aTree.getMembers()
```
##### A partir d'un noeud
```python
proteinList = aTree.getMembersByName("transmembrane transport")
proteinList = aTree.getMembersByID("GO:0055085")
```
Attention, cette liste étant redondante on prendra soin de rendre ses éléments uniques avec :
```python
proteinListNonRedundant = list( set(aTree.getMembersByName("transmembrane transport")) )
proteinListNonRedundant = list( set(aTree.getMembersByID("GO:0055085")) )
```
#### Acceder directement à un noeud
```python
goTerm = aTree.getByName("transmembrane transport")
goTerm = aTree.getByID("GO:0055085")
```

#### Elager un arbre
Retirer tous les noeuds n'ayant aucun des éléments suivants parmi leurs protéines annotées

```python
def predicat(goTerm):
    return set(goTerm.getMembers()) & set(['P75936', 'P76231', 'P0A8S9'])

bTree = aTree.drop(predicat)
```

## Evaluer la pValue d'enrichissment d'un terme GO parmi une liste de protéine donnée

Avant d'appeler `righEnd_pValue`, on devra donc determiner les valeurs de ses paramètres $N,K,n,k$

1. Lister les protéines surabondantes
2. Lister toutes les protéines sous le terme GO de votre choix 
    1. dans l'experience
    2. dans le protéome
3. Calculer l'intersection entre *A* et *B*

###### *Aide:*
Vous vous aiderez 
- de l'API de `go.annotationTree` 
- des commandes [`set` de python](https://docs.python.org/3/library/stdtypes.html#set)
- du [filtrage de colonne Pandas](https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/) pour évaluer la liste de protéines surabondantes
- de la [fonction subplot de matplotlib](https://stackoverflow.com/questions/42818361/how-to-make-two-plots-side-by-side-using-python), pour dessiner les deux courbes cote-à-cote


In [ ]:
### Cellule Exercice

## Cas de mise en forme
### Analyse de la surreprésentation

Le package *stat_utils.py* permet d'appliquer l'analyse de surreprésentation récursivement, en profondeur, à partir d'un terme GO racine.

#### Estimation récursive des enrichissements

* Choix du terme GO parent à partir duquel l'analyse ORA sera recursivement appliquée
```python
pathWayRoot = xpGoTree.getByName("transmembrane transport")
```

* Définition du terme GO regroupant **tout le protéome**
```python
pathWayBKG = fullEcoliGoTree.getByName("biological process")
```

* Calcul de l'enrichissement en termes GO successifs parmi les protéines surabondantes (ici, *saList*)
```python
oraFisher, oraCDF = computeORA_BKG(pathWayRoot, saList, pathWayBKG)
```
* Des arbres peuvent également être passés.
```python
tm_transport_GoTree = xpGoTree.newRoot("transmembrane transport")
oraFisher, oraCDF = computeORA_BKG(tm_transport_GoTree, saList, fullEcoliGoTree)
```

#### Affichage des enrichissements
La fonction ci-dessous va vous permettre d'afficher les pathway par ordre croissant de p-value
```python
def printRankings(saList,oraFisher, oraCDF):
    fisherRanking = [ ( d[0], d[1].name, len( set(d[1].getMembers()) & set(saList) ), len(saList) ) for d in sorted(oraFisher, key=lambda x:x[0]) ]
    cdfRanking = [ ( d[0], d[1].name, len( set(d[1].getMembers()) & set(saList) ), len(saList) ) for d in sorted(oraCDF, key=lambda x:x[0]) ]
    print(f"FISHER test Rankings:\n{fisherRanking}")
    print('\n---\n')
    print(f"CDF Rankings:\n{cdfRanking}")
```

##### Intégration des résultats à la visualisation du notebook 

En vous inspirant de la fonction ci-dessus, proposer une classe `Rankings` implémentant une méthode [`_repr_html_`](https://ipython.readthedocs.io/en/stable/config/integrating.html) qui permette un affichage plus lisible des résultats, comme par exemple:

![caption](../figs/_repr_html_.png)

###### Squelette de la classe de visualisation
```python
class Rankings():
    def __init__(self, oraResults, proteinList, title="Pathway ORA"):
        self.title = title
        self.ranks = [ ( d[0], d[1].name, len( set(d[1].getMembers()) & set(proteinList) ), len(proteinList) ) for d in sorted(oraResults, key=lambda x:x[0]) ]
    def _repr_html_(self):
        pass
```


#### Appliquer cette analyse à tous les termes GO du namespace de votre choix, "biological process", "molecular function", "cellular component"
 - Vous pouvez en encapsuler les différentes étapes dans une fonction de ce type

```python
    def computePathway(goTermString, xpTreeTerm, bkgTreeTerm, uniprotIdList) -> Rankings
```



In [ ]:
### Cellule Exercice

## Serialisation des résultats
Afin de faciliter le travail des séances suivantes, vous devez produire une représentation de l'objet Rankings que vous venez de construire au [format JSON](https://en.wikipedia.org/wiki/JSON).
Vous vous limiterez à son attribut ranks.
* Inspectez cet attribut.
* A l'aide du module [json](https://docs.python.org/3/library/json.html), ecrivez le contenu de ranks dans un fichier TP_ORA.json

In [2]:
### Cellule Exercice